In [3]:
%matplotlib inline

In [2]:
# Deal with data
import pandas as pd
import numpy as np
import json
from glob import glob
import os

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML

In [4]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [23]:
front_key = 'fga-eps-mds_2021.2-Oraculo-FrontEnd'
profile_key = 'fga-eps-mds_2021.2-Oraculo-Profile'
registros_key = 'fga-eps-mds_2021.2-Oraculo-Registros'

In [19]:
metric_list = ['files',
               'functions',
               'complexity',
               'comment_lines_density',
               'duplicated_lines_density',
               'security_rating',
               'tests',
               'test_success_density',
               'test_execution_time',
               'reliability_rating']

len(metric_list)

10

In [20]:
def generate_metric_string(metric_list):
    metric_str = ''
    
    for metric in metric_list:
        metric_str += metric + ','
        
    return metric_str

In [21]:
metric_str = generate_metric_string(metric_list)

In [24]:
front_url = f"https://sonarcloud.io/api/measures/component_tree?component={front_key}&metricKeys={metric_str}&ps=500"
profile_url = f"https://sonarcloud.io/api/measures/component_tree?component={profile_key}&metricKeys={metric_str}&ps=500"
registros_url = f"https://sonarcloud.io/api/measures/component_tree?component={registros_key}&metricKeys={metric_str}&ps=500"

In [25]:
http = urllib3.PoolManager()

In [26]:
front_request = http.request('GET', front_url)
front_request.status

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200

In [29]:
profile_request = http.request('GET', profile_url)
profile_request.status

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200

In [30]:
registros_request = http.request('GET', registros_url)
registros_request.status

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200

In [31]:
front_json = json.loads(front_request.data.decode('utf-8'))
profile_json = json.loads(profile_request.data.decode('utf-8'))
registros_json = json.loads(registros_request.data.decode('utf-8'))

In [33]:
project_front_json = front_json['baseComponent']['measures']
project_profile_json = profile_json['baseComponent']['measures']
project_registros_json = registros_json['baseComponent']['measures']

In [34]:
project_front_data = pd.DataFrame(project_front_json)
project_profile_data = pd.DataFrame(project_profile_json)
project_registros_data = pd.DataFrame(project_registros_json)

In [35]:
project_front_data

,metric,value,bestValue
0,duplicated_lines_density,0.0,True
1,functions,54,NaN
2,security_rating,1.0,True
3,files,18,NaN
4,complexity,122,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,2.7,False


In [36]:
project_profile_data

,metric,value,bestValue
0,duplicated_lines_density,0.0,True
1,functions,30,NaN
2,security_rating,1.0,True
3,files,11,NaN
4,complexity,61,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,0.6,False


In [37]:
project_registros_data

,metric,value,bestValue
0,duplicated_lines_density,0.0,True
1,functions,54,NaN
2,security_rating,1.0,True
3,files,18,NaN
4,complexity,122,NaN
5,reliability_rating,1.0,True
6,comment_lines_density,2.7,False


In [38]:
def metric_per_file(json):
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

In [39]:
front_file_json = metric_per_file(front_json)
profile_file_json = metric_per_file(profile_json)
registros_file_json = metric_per_file(registros_json)

In [40]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df = df.rename({'index': 'path'}, axis=1).drop(['files'], axis=1)

    return df

In [41]:
front_files_data = generate_file_dataframe(metric_list, front_file_json, language_extension = 'js')
profile_files_data = generate_file_dataframe(metric_list, profile_file_json, language_extension = 'js')
registros_files_data = generate_file_dataframe(metric_list, registros_file_json, language_extension = 'js')


In [42]:
front_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/Model/Department.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/Controller/DepartmentController.js,2,3,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Constants/errors.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/Model/Field.js,1,1,40.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Model/History.js,2,2,9.4,0.0,1.0,NaN,100.0,NaN,1.0
5,src/index.js,1,3,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/Database/index.js,7,13,6.8,0.0,1.0,NaN,100.0,NaN,1.0
7,src/Model/Record.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
8,src/Controller/RecordController.js,24,74,1.1,0.0,1.0,NaN,100.0,NaN,1.0
9,src/Model/RecordNumber.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0


In [43]:
profile_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/Model/Department.js,3,3,0.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/Controller/DepartmentController.js,3,6,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Utils/hash.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/index.js,0,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Database/index.js,7,12,1.4,0.0,1.0,NaN,100.0,NaN,1.0
5,src/Utils/JWT.js,2,4,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/Model/Level.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
7,src/routes.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
8,src/Model/User.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
9,src/Controller/UserController.js,10,30,0.9,0.0,1.0,NaN,100.0,NaN,1.0


In [44]:
registros_files_data

,path,functions,complexity,comment_lines_density,duplicated_lines_density,security_rating,tests,test_success_density,test_execution_time,reliability_rating
0,src/Model/Department.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
1,src/Controller/DepartmentController.js,2,3,0.0,0.0,1.0,NaN,100.0,NaN,1.0
2,src/Constants/errors.js,0,0,0.0,0.0,1.0,NaN,100.0,NaN,1.0
3,src/Model/Field.js,1,1,40.0,0.0,1.0,NaN,100.0,NaN,1.0
4,src/Model/History.js,2,2,9.4,0.0,1.0,NaN,100.0,NaN,1.0
5,src/index.js,1,3,0.0,0.0,1.0,NaN,100.0,NaN,1.0
6,src/Database/index.js,7,13,6.8,0.0,1.0,NaN,100.0,NaN,1.0
7,src/Model/Record.js,2,2,0.0,0.0,1.0,NaN,100.0,NaN,1.0
8,src/Controller/RecordController.js,24,74,1.1,0.0,1.0,NaN,100.0,NaN,1.0
9,src/Model/RecordNumber.js,1,1,0.0,0.0,1.0,NaN,100.0,NaN,1.0


In [45]:
def m1(df):
    
    density_non_complex_files = round((len(df[(df['complexity'].astype(float)/df['functions'].astype(float)) < 10])/len(df))*100, 2)
    
    return density_non_complex_files

In [46]:
def m2(df):
    
    density_comment_files = round((len(df[(df['comment_lines_density'].astype(float) > 10) & (df['comment_lines_density'].astype(float) < 30)])/len(df))*100, 2)
    
    return density_comment_files

In [47]:
def m3(df):
    
    duplication = round((len(df[(df['duplicated_lines_density'].astype(float) < 5)])/len(df))*100, 2)
    
    return duplication

In [48]:
def m4(df):
    
    non_blocking_files = round((len(df[(df['security_rating'].astype(float) >= 4)])/len(df))*100,2)
    
    return non_blocking_files

In [49]:
def m5(df):
    
    test_success_file = df[['path', 'test_success_density']]
    test_success_repository = df['test_success_density'].astype(float).mean()
    
    print("Project test unit density: ", test_success_repository)
    
    return test_success_file

In [50]:
def m6(df):
    
    fast_test_df = df[(df['test_execution_time'].astype(float) < 300)]
    fast_test_df['fast_test'] = fast_test_df['test_execution_time']/fast_test_df['tests']
    
    fast_test_file = fast_test_df[['path', 'fast_test']]
    
    fast_test_repository = fast_test_df['fast_test'].astype(float).mean()
    
    print("Project test unit density: ", fast_test_repository)
    
    return fast_test_file

In [51]:
m1(profile_files_data)

80.0

In [52]:
m5(profile_files_data)

Project test unit density:  100.0


,path,test_success_density
0,src/Model/Department.js,100.0
1,src/Controller/DepartmentController.js,100.0
2,src/Utils/hash.js,100.0
3,src/index.js,100.0
4,src/Database/index.js,100.0
5,src/Utils/JWT.js,100.0
6,src/Model/Level.js,100.0
7,src/routes.js,100.0
8,src/Model/User.js,100.0
9,src/Controller/UserController.js,100.0


In [53]:
m5(registros_files_data)

Project test unit density:  100.0


,path,test_success_density
0,src/Model/Department.js,100.0
1,src/Controller/DepartmentController.js,100.0
2,src/Constants/errors.js,100.0
3,src/Model/Field.js,100.0
4,src/Model/History.js,100.0
5,src/index.js,100.0
6,src/Database/index.js,100.0
7,src/Model/Record.js,100.0
8,src/Controller/RecordController.js,100.0
9,src/Model/RecordNumber.js,100.0
